In [30]:
#import modules
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import time

#FUNCTIONS
#function that takes desired variables with your list of dictionaries scraped and returns a clean list of these variables
def get_vars(stats, vars_wanted):
    stats_vars = []
    items=[]
    #iterate through scraped data to find desired variables
    for list_item in stats:
        dict_item = [value for key,value in list_item.items()]
        if dict_item[0] in vars_wanted:
            #check for duplicates
            if dict_item not in items:
                items.append(dict_item)
                stats_vars.append(list_item)
    return stats_vars


#---------------------------------------------------------------------------------------------------------------------------
#MAIN

#URL of page to be scraped
url = 'https://www.pgatour.com/players.html'

#Retrieve page with the requests module
response = requests.get(url)

#create BeautifulSoup object; parse w 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')


#SCRAPE LINK TO EACH PLAYER STAT PAGE***********************************
#get names of all the player links
#retrieve the parent divs for all links
players = soup.find_all('span',class_="name")

#create empty list to accept data
player_names = []
player_url = []

#loop through each parent div and grab the link to the player stat page
for player in players:
    #get name of player
    player_names.append(player.a.text)
    #get url for player performance page
    player_url.append(player.a['href'])


#SCRAPE PGA PERFORMANCE DATA FOR EACH INDIVIDUAL PLAYER******************
base_url = 'https://www.pgatour.com'

#execute chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

#master data set
DATA = []
#keep track of player count
player_count = 0
#keep track of the players unable to scrape
error_player = []

for player,url in zip(player_names,player_url):
    scrape_url = base_url + url

    #SPLINTER
    browser.visit(scrape_url)

    try:
        browser.click_link_by_partial_text("Performance")
        time.sleep(1)
    
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')   

        #photo of player
        try:
            photo_url = soup.find('img', class_='photo')['src']
            player_intro = [{'Player Name': player, 'photo_url': photo_url}]
        except TypeError:
            player_intro = [{'Player Name': player, 'photo_url': 'none'}]

        #scrape all html hstat code
        info = soup.findAll('div', class_='item')

        #headline stats
        info_stats = []

        for i in info:
            try:
                #get stat name
                caption = i.find('div',class_='denotation').text
                #get stat value
                value = i.find('div', class_='value').text
                #clean up text
                value = value.replace('\xa0','')
                value = value.replace('\n','')
                #save to dictionary
                post = {'caption':caption, 'value': value}
                #append to list
                info_stats.append(post)
            except AttributeError:
                nothing = 0
        
        #scrape all html hstat code
        hstats = soup.findAll('div', class_='stat')

        #headline stats
        h_stats = []
        for hstat in hstats:
            try:
                #get stat name
                caption = hstat.find('div',class_='caption').text
                #get stat value
                value = hstat.find('div',class_='value').text
                
                #save to dictionary
                post = {'caption':caption, 'value': value}

                #append to list
                h_stats.append(post)
            except AttributeError:
                nothing = 0

        #scrape all html astat code
        astats = soup.findAll('tr')

        #attribute stats
        a_stats = []

        for astat in astats:
            try:
                #get the stat name
                caption = astat.find('td',class_='caption').text
                #get stat value
                value = astat.find('td',class_='value').text
                
                #save to dictionary
                post = {'caption':caption, 'value': value}
                
                #append to list
                a_stats.append(post)
            except AttributeError:
                nothing = 0

        #scrape for additional needed info
        extrastats = soup.findAll('td')

        #attribute stats
        extra_stats = []

        for extra in extrastats:
            try:
                #get the stat name
                text = extra.text
                #append to list
                extra_stats.append(text)
            except AttributeError:
                nothing=0

        
        #these attributes are unique with no captions/values-All string format
        #search sub_strings of desired variables for values
        sub_strings = ['Total Left rough', 'Total Right rough', 'Possible Fwys', 'Distance Rank', 'Accuracy Rank',
            'Total Club Head Speed', 'Total Attempts']

        extra_stats_var= []
        for sub in sub_strings:
            x = [s for s in extra_stats if sub in s]
            if x:
                x = x[0].split(':')
                x[1] = x[1].replace(' ','')

                post = {'caption': x[0], 'value': x[1]}
                extra_stats_var.append(post)
            else:
                nothing=0

        #variables wanted from info stats
        info_var = ['Height', 'Weight', 'AGE', 'Turned Pro', 'College', 'Birthplace' ,'FEDEXCUP Rank', 'FEDEXCUP Points', 'Scoring Average']
        #variables wanted from headline stats
        h_var = ['Total Distance', 'Total Drives', '# of Drives', 'Fairways Hit', 'Possible Fairways', 'Measured Rounds']
        #variables wanted from additional stats
        a_var = ['Driving Distance','Driving Accuracy Percentage','Total Driving','Club Head Speed',
                'Distance from Edge of Fairway','Left Rough Tendency','Right Rough Tendency','Total Driving Efficiency']
        extra_var = ['Total Left rough', 'Total Right rough', 'Possible Fwys', 'Distance Rank', 'Accuracy Rank',
            'Total Club Head Speed', 'Total Attempts']

        info_stats_vars = get_vars(info_stats, info_var)
        h_stats_vars = get_vars(h_stats, h_var)
        a_stats_vars = get_vars(a_stats, a_var)
        extra_stats_vars = get_vars(extra_stats_var, sub_strings)

        #combine all stat variables
        all_stat_var = info_stats_vars + h_stats_vars + a_stats_vars + extra_stats_vars

        bla = soup.findAll('div', class_ = 'holder')
        dates_ = soup.findAll('td', class_ = 'date')
        rounds = soup.findAll('td', class_='round')

        tourney_name = []
        all_text = []
        scores = []
        to_par = []
        pos = []
        dates = []

        for i in bla:
            x = i.find('tbody')
            #tourney info
            tourneys = x.findAll('p')
            #need this for all text
            tds = x.findAll('td')
            #get all text to use later for pos
            [all_text.append(td.text) for td in tds]
            
            #tournament names
            [tourney_name.append(j.text) for j in tourneys]

        #clean dates
        [dates.append(d.text) for d in dates_]
        #scores of each round in increments of 4 ('--' means no score)
        [scores.append(r.text) for r in rounds]
        #now append tournament position results by getting list item after tournament name
        [pos.append(all_text[all_text.index(tourney)+1]) for tourney in tourney_name]
        #now append tournament position results by getting list item after tournament name
        [to_par.append(all_text[all_text.index(tourney)+8]) for tourney in tourney_name]

        #create final dictionary of tournaments for the past year
        tournament_history = []
        for date,tourney,score,rank in zip(dates,tourney_name,to_par,pos):
            try:
                #create dictionary with all info
                post = {'Date':date, 
                        'Tournament Name':tourney, 
                        'Total Score':score, 
                        'POS':rank}
                #append to final list
                tournament_history.append(post)
            except AttributeError:
                nothing=0

        #output list
        player_final = {}
        player_final['player_intro'] = player_intro
        player_final['all_stat_var'] = all_stat_var
        player_final['tournament_hist'] = tournament_history

        DATA.append(player_final)
        player_count += 1
        print(f"Player Data Successfully Acquired!: {player}")
    
    except:
        error_player.append(player_count)
        print(f"!!!!!!!!!!!!!!Data Retrieval Unsuccessful: {player}")
        
browser.quit()

Player Data Successfully Acquired!: Adams, John
Player Data Successfully Acquired!: Adams, Sam
Player Data Successfully Acquired!: Aguilar, Felipe
Player Data Successfully Acquired!: Albertson, Anders
Player Data Successfully Acquired!: Aldridge, Tyler
Player Data Successfully Acquired!: Alker, Steven
Player Data Successfully Acquired!: Allem, Fulton
Player Data Successfully Acquired!: Allen, Michael
Player Data Successfully Acquired!: Allenby, Robert
Player Data Successfully Acquired!: Ames, Stephen
Player Data Successfully Acquired!: An, Byeong Hun
Player Data Successfully Acquired!: Ancer, Abraham
Player Data Successfully Acquired!: Andersson Hed, Fredrik
Player Data Successfully Acquired!: Andrade, Billy
Player Data Successfully Acquired!: Aoki, Isao
Player Data Successfully Acquired!: Aphibarnrat, Kiradech
Player Data Successfully Acquired!: Appleby, Stuart
Player Data Successfully Acquired!: Armour, Ryan
Player Data Successfully Acquired!: Armour III, Tommy
Player Data Successful

Player Data Successfully Acquired!: Dobyns, Matt
Player Data Successfully Acquired!: Dodds, Trevor
Player Data Successfully Acquired!: Dodt, Andrew
Player Data Successfully Acquired!: Donald, Luke
Player Data Successfully Acquired!: Donald, Mike
Player Data Successfully Acquired!: Donaldson, Jamie
Player Data Successfully Acquired!: Dougherty, Ed
Player Data Successfully Acquired!: Dougherty, Sean
Player Data Successfully Acquired!: Douglass, Dale
Player Data Successfully Acquired!: Dredge, Bradley
Player Data Successfully Acquired!: Dubuisson, Victor
Player Data Successfully Acquired!: Dufner, Jason
Player Data Successfully Acquired!: Duke, Ken
Player Data Successfully Acquired!: Duncan, Tyler
Player Data Successfully Acquired!: Dunne, Paul
Player Data Successfully Acquired!: Durant, Joe
Player Data Successfully Acquired!: Duval, David
Player Data Successfully Acquired!: Dyson, Simon
Player Data Successfully Acquired!: DÃ­az, Roberto
Player Data Successfully Acquired!: Eastwood, Bob
P

Player Data Successfully Acquired!: Huston, John
Player Data Successfully Acquired!: Ikeda, Yuta
Player Data Successfully Acquired!: Ilonen, Mikko
Player Data Successfully Acquired!: Im, Sungjae
Player Data Successfully Acquired!: Imada, Ryuji
Player Data Successfully Acquired!: Immelman, Trevor
Player Data Successfully Acquired!: Inman, Joe
Player Data Successfully Acquired!: Inman, John
Player Data Successfully Acquired!: Irawan, Arie Ahmad
Player Data Successfully Acquired!: Irwin, Hale
Player Data Successfully Acquired!: Iverson, Don
Player Data Successfully Acquired!: Jacklin, Tony
Player Data Successfully Acquired!: Jacobs, Tommy
Player Data Successfully Acquired!: Jacobsen, Peter
Player Data Successfully Acquired!: Jacobson, Freddie
Player Data Successfully Acquired!: Jaeckel, Barry
Player Data Successfully Acquired!: Jaeger, Stephan
Player Data Successfully Acquired!: Jaidee, Thongchai
Player Data Successfully Acquired!: Jamieson, Jim
Player Data Successfully Acquired!: Jamieso

Player Data Successfully Acquired!: McLendon, Mac
Player Data Successfully Acquired!: McNeill, George
Player Data Successfully Acquired!: McRoy, Spike
Player Data Successfully Acquired!: Mediate, Rocco
Player Data Successfully Acquired!: Meesawat, Prom
Player Data Successfully Acquired!: Merrick, John
Player Data Successfully Acquired!: Merritt, Troy
Player Data Successfully Acquired!: Micheel, Shaun
Player Data Successfully Acquired!: Mickelson, Phil
Player Data Successfully Acquired!: Middlecoff, Cary
Player Data Successfully Acquired!: Miller, Allen
Player Data Successfully Acquired!: Miller, Johnny
Player Data Successfully Acquired!: Mitchell, Jeff
Player Data Successfully Acquired!: Mitchell, Keith
Player Data Successfully Acquired!: Mize, Larry
Player Data Successfully Acquired!: Molder, Bryce
Player Data Successfully Acquired!: Molinari, Edoardo
Player Data Successfully Acquired!: Molinari, Francesco
Player Data Successfully Acquired!: Moore, Jonathan
Player Data Successfully Ac

Player Data Successfully Acquired!: Senden, John
Player Data Successfully Acquired!: Sharma, Shubhankar
Player Data Successfully Acquired!: Shaw, Tom
Player Data Successfully Acquired!: Shearer, Bob
Player Data Successfully Acquired!: Shih-chang, Chan
Player Data Successfully Acquired!: Short, Jr., Wes
Player Data Successfully Acquired!: Sieckmann, Tom
Player Data Successfully Acquired!: Siem, Marcel
Player Data Successfully Acquired!: Sifford, Charles
Player Data Successfully Acquired!: Sikes, R.H.
Player Data Successfully Acquired!: Sills, Tony
Player Data Successfully Acquired!: Silverman, Ben
Player Data Successfully Acquired!: Simpson, Scott
Player Data Successfully Acquired!: Simpson, Tim
Player Data Successfully Acquired!: Simpson, Webb
Player Data Successfully Acquired!: Sindelar, Joey
Player Data Successfully Acquired!: Singh, Jeev Milkha
Player Data Successfully Acquired!: Singh, Vijay
Player Data Successfully Acquired!: Slattery, Lee
Player Data Successfully Acquired!: Sloan

In [41]:
DATA[803]

{'player_intro': [{'Player Name': 'Woods,\xa0Tiger',
   'photo_url': 'https://pga-tour-res.cloudinary.com/image/upload/c_fill,d_headshots_default.png,f_auto,g_face:center,h_350,q_auto,w_280/headshots_08793.png'}],
 'all_stat_var': [{'caption': 'Height', 'value': '6 ft, 1 in'},
  {'caption': 'Weight', 'value': '185lbs'},
  {'caption': 'AGE', 'value': '43'},
  {'caption': 'Turned Pro', 'value': '1996'},
  {'caption': 'College', 'value': 'Stanford University '},
  {'caption': 'Birthplace', 'value': 'Cypress, California'},
  {'caption': 'FEDEXCUP Rank', 'value': '17'},
  {'caption': 'FEDEXCUP Points', 'value': '888'},
  {'caption': 'Scoring Average', 'value': '69.729'},
  {'caption': 'Measured Rounds', 'value': '15'},
  {'caption': 'Total Distance', 'value': '11,984'},
  {'caption': 'Total Drives', 'value': '40'},
  {'caption': 'Total Distance', 'value': '59,758'},
  {'caption': '# of Drives', 'value': '210'},
  {'caption': 'Fairways Hit', 'value': '181'},
  {'caption': 'Possible Fairways'

## MySQL Data Storage

In [33]:
import MySQLdb

In [36]:
#connect to database
db = MySQLdb.connect("localhost","root","titleist","pga_data" )
#prepare a cursor object
cursor = db.cursor()

In [72]:
#test to insert
test = DATA[803]
test2 = DATA[802]

In [40]:
test

{'player_intro': [{'Player Name': 'Woods,\xa0Tiger',
   'photo_url': 'https://pga-tour-res.cloudinary.com/image/upload/c_fill,d_headshots_default.png,f_auto,g_face:center,h_350,q_auto,w_280/headshots_08793.png'}],
 'all_stat_var': [{'caption': 'Height', 'value': '6 ft, 1 in'},
  {'caption': 'Weight', 'value': '185lbs'},
  {'caption': 'AGE', 'value': '43'},
  {'caption': 'Turned Pro', 'value': '1996'},
  {'caption': 'College', 'value': 'Stanford University '},
  {'caption': 'Birthplace', 'value': 'Cypress, California'},
  {'caption': 'FEDEXCUP Rank', 'value': '17'},
  {'caption': 'FEDEXCUP Points', 'value': '888'},
  {'caption': 'Scoring Average', 'value': '69.729'},
  {'caption': 'Measured Rounds', 'value': '15'},
  {'caption': 'Total Distance', 'value': '11,984'},
  {'caption': 'Total Drives', 'value': '40'},
  {'caption': 'Total Distance', 'value': '59,758'},
  {'caption': '# of Drives', 'value': '210'},
  {'caption': 'Fairways Hit', 'value': '181'},
  {'caption': 'Possible Fairways'

In [ ]:
sql = """CREATE TABLE JOEY(
FIRST_NAME CHAR(20) NOT NULL,
LAST_NAME CHAR(20),
AGE INT,
SEX CHAR(1),
INCOME FLOAT
)"""

## MongoDB Data Storage

In [228]:
import pymongo
from bson import ObjectId

In [57]:
# Create connection variable
conn = 'mongodb://localhost:27017'

In [58]:
# Pass connection to the pymongo instance.
client = pymongo.MongoClient(conn)

In [59]:
# Connect to a database. Will create one if not already available.
db = client.pga_data

In [60]:
# create mongo connection
collection = db.data

In [96]:
test4 = DATA[5]

In [109]:
x = db.collection.insert_one(test)

In [128]:
x.inserted_id

ObjectId('5cd313a13410bbfec9646ab0')

works

In [229]:
bla = db.collection.find_one({'_id':ObjectId('5cd314fd3410bbfec9646ab1')})

In [230]:
bla

{'_id': ObjectId('5cd314fd3410bbfec9646ab1'),
 'player_intro': [{'Player Name': 'Alker,\xa0Steven',
   'photo_url': 'https://pga-tour-res.cloudinary.com/image/upload/c_fill,d_headshots_default.png,f_auto,g_face:center,h_350,q_auto,w_280/headshots_20771.png'}],
 'all_stat_var': [{'caption': 'Height', 'value': '5 ft, 11 in'},
  {'caption': 'Weight', 'value': '155lbs'},
  {'caption': 'AGE', 'value': '47'},
  {'caption': 'Turned Pro', 'value': '1995'},
  {'caption': 'College', 'value': 'New Zealand education, college, 7th form '},
  {'caption': 'Birthplace', 'value': 'Hamilton, New Zealand'},
  {'caption': 'Measured Rounds', 'value': '51'},
  {'caption': 'Total Distance', 'value': '34,884'},
  {'caption': 'Total Drives', 'value': '126'},
  {'caption': 'Total Distance', 'value': '194,486'},
  {'caption': '# of Drives', 'value': '714'},
  {'caption': 'Fairways Hit', 'value': '595'},
  {'caption': 'Possible Fairways', 'value': '881'},
  {'caption': 'Driving Distance', 'value': '276.9'},
  {'c

Works

In [199]:
test_data = db.collection.find()

In [204]:
for bla in range(0,3):
    print(test_data[bla]['player_intro'])

[{'Player Name': 'Alker,\xa0Steven', 'photo_url': 'https://pga-tour-res.cloudinary.com/image/upload/c_fill,d_headshots_default.png,f_auto,g_face:center,h_350,q_auto,w_280/headshots_20771.png'}]
[{'Player Name': 'Woods,\xa0Tiger', 'photo_url': 'https://pga-tour-res.cloudinary.com/image/upload/c_fill,d_headshots_default.png,f_auto,g_face:center,h_350,q_auto,w_280/headshots_08793.png'}]
[{'Player Name': 'Woodland,\xa0Gary', 'photo_url': 'https://pga-tour-res.cloudinary.com/image/upload/c_fill,d_headshots_default.png,f_auto,g_face:center,h_350,q_auto,w_280/headshots_31323.png'}]


In [189]:
test_data[0]['all_stat_var']

[{'caption': 'Height', 'value': '5 ft, 11 in'},
 {'caption': 'Weight', 'value': '155lbs'},
 {'caption': 'AGE', 'value': '47'},
 {'caption': 'Turned Pro', 'value': '1995'},
 {'caption': 'College', 'value': 'New Zealand education, college, 7th form '},
 {'caption': 'Birthplace', 'value': 'Hamilton, New Zealand'},
 {'caption': 'Measured Rounds', 'value': '51'},
 {'caption': 'Total Distance', 'value': '34,884'},
 {'caption': 'Total Drives', 'value': '126'},
 {'caption': 'Total Distance', 'value': '194,486'},
 {'caption': '# of Drives', 'value': '714'},
 {'caption': 'Fairways Hit', 'value': '595'},
 {'caption': 'Possible Fairways', 'value': '881'},
 {'caption': 'Driving Distance', 'value': '276.9'},
 {'caption': 'Driving Accuracy Percentage', 'value': '67.54%'},
 {'caption': 'Distance from Edge of Fairway', 'value': '25\' 1"'},
 {'caption': 'Left Rough Tendency', 'value': '10.67%'},
 {'caption': 'Right Rough Tendency', 'value': '15.03%'},
 {'caption': 'Total Driving', 'value': '204'},
 {'ca